In [1]:
# Import the standard packages normally utilize by Data Scientist
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans
from sklearn import preprocessing
from collections import Counter
import scipy.sparse as sparse
import gensim
import spacy

In [3]:
data=pd.read_excel("/content/movie_data.xlsx")
data.info()
data.rename(columns={'Unnamed: 0': 'movie_id','Movie Name':'Title'}, inplace=True)

columns=['Cast','Director','Genre','Title','Description']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1000 non-null   int64  
 1   Movie Name          1000 non-null   object 
 2   Year of Release     997 non-null    object 
 3   Watch Time          948 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        938 non-null    float64
 6   Metascore of movie  899 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                998 non-null    object 
 9   Votes               938 non-null    object 
 10  Description         1000 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 86.1+ KB


In [4]:
data[columns].isnull().values.any()

True

In [5]:
def get_important_features(data):
    important_features=[]
    for i in range (0,data.shape[0]):
        important_features.append(data['Title'][i]+' '+data['Director'][i]+' '+data['Genre'][i]+' '+data['Description'][i]+' '+data['Cast'])
    return important_features

In [7]:
data['important_features']=get_important_features(data)

In [8]:
data

,movie_id,Title,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description,important_features
0,0,The Flash,2023,144 min,"\nAction, Adventure, Fantasy",7.2,56.0,Andy Muschietti,"Ezra Miller,Michael Keaton,Sasha Calle,Michael...","67,412",Barry Allen uses his super speed to change the...,"0 The Flash Andy Muschietti \nAction, Adv..."
1,1,Extraction II,2023,122 min,"\nAction, Thriller",7.1,57.0,Sam Hargrave,"Chris Hemsworth,Golshifteh Farahani,Adam Bessa...","59,842",After barely surviving his grievous wounds fro...,"0 Extraction II Sam Hargrave \nAction, Th..."
2,2,Spider-Man: Across the Spider-Verse,2023,140 min,"\nAnimation, Action, Adventure",8.9,86.0,Joaquim Dos Santos,"Kemp Powers,Justin K. Thompson,Shameik Moore,H...","152,594","Miles Morales catapults across the Multiverse,...",0 Spider-Man: Across the Spider-Verse Joa...
3,3,No Hard Feelings,2023,103 min,\nComedy,6.8,58.0,Gene Stupnitsky,"Jennifer Lawrence,Andrew Barth Feldman,Laura B...","6,204","On the brink of losing her home, Maddie finds ...",0 No Hard Feelings Gene Stupnitsky \nCome...
4,4,Asteroid City,2023,105 min,"\nComedy, Drama, Romance",7.1,75.0,Wes Anderson,"Jason Schwartzman,Scarlett Johansson,Tom Hanks...","13,287",Following a writer on his world famous fiction...,"0 Asteroid City Wes Anderson \nComedy, Dr..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Jumanji: The Next Level,2019,123 min,"\nAction, Adventure, Comedy",6.7,58.0,Jake Kasdan,"Dwayne Johnson,Jack Black,Kevin Hart,Karen Gillan","265,121","In Jumanji: The Next Level, the gang is back b...",0 Jumanji: The Next Level Jake Kasdan \nA...
996,996,Star Trek: The Motion Picture,1979,143 min,"\nAdventure, Mystery, Sci-Fi",6.4,50.0,Robert Wise,"William Shatner,Leonard Nimoy,DeForest Kelley,...","93,544",When an alien spacecraft of enormous power is ...,0 Star Trek: The Motion Picture Robert Wi...
997,997,Thirteen Lives,2022,147 min,"\nAction, Adventure, Biography",7.8,66.0,Ron Howard,"Viggo Mortensen,Colin Farrell,Joel Edgerton,To...","61,369",A rescue mission is assembled in Thailand wher...,"0 Thirteen Lives Ron Howard \nAction, Adv..."
998,998,Star Wars: Episode III - Revenge of the Sith,2005,140 min,"\nAction, Adventure, Fantasy",7.6,68.0,George Lucas,"Hayden Christensen,Natalie Portman,Ewan McGreg...","814,338","Three years into the Clone Wars, Obi-Wan pursu...",0 Star Wars: Episode III - Revenge of the...


In [19]:
tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')
data['important_features'] = data['important_features'].astype('str').replace(r'\d+', '')
data['important_features']=data['important_features'].map(lambda x: x.lower())
tfidf_matrix = tfidf.fit_transform(data['important_features'])
tfidf_matrix.shape

(1000, 1)

In [20]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(data.index, index=data['Title']).drop_duplicates()

In [27]:
# Recommendation function
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Title'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df

In [22]:
get_recommendations('Spider-Man: Far from Home')

,Movies,id
0,Extraction II,1
1,Spider-Man: Across the Spider-Verse,2
2,No Hard Feelings,3
3,Asteroid City,4
4,Challengers,5


In [28]:
get_recommendations('Fast X')

,Movies,id
0,Extraction II,1
1,Spider-Man: Across the Spider-Verse,2
2,No Hard Feelings,3
3,Asteroid City,4
4,Challengers,5


In [29]:
data.info()
new = data.drop(columns=['Year of Release','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   movie_id             1000 non-null   int64  
 1   Title                1000 non-null   object 
 2   Year of Release      997 non-null    object 
 3   Watch Time           948 non-null    object 
 4   Genre                1000 non-null   object 
 5   Movie Rating         938 non-null    float64
 6   Metascore of movie   899 non-null    float64
 7   Director             1000 non-null   object 
 8   Cast                 998 non-null    object 
 9   Votes                938 non-null    object 
 10  Description          1000 non-null   object 
 11  important_features   1000 non-null   object 
 12  important_features1  0 non-null      float64
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB


In [30]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('similarity.pkl','wb'))

In [ ]:
# Designing the Front  End

In [31]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 31.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=ab797758e90dc051e23927892380392f154a5949e6801a07016493e7a8cdf05c
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  

In [37]:

import pickle
import streamlit as st

def recommend(movie):
    index = movies[movies['Title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    for i in distances[1:6]:
        recommended_movie_names.append(movies.iloc[i[0]].Title)

    return recommended_movie_names

page_bg_img = '''
<style>
      .stApp {
  background-image: url("https://payload.cargocollective.com/1/11/367710/13568488/MOVIECLASSICSerikweb_2500_800.jpg");
  background-size: cover;
}
</style>
'''

st.markdown(page_bg_img, unsafe_allow_html=True)


st.markdown('# Movie Recommendation System')
movies = pickle.load(open('movie_list.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))

movie_list = movies['Title'].values
selected_movie = st.selectbox(
    "Type or select a movie from the dropdown",
    movie_list
)


if st.button('Show Recommendation'):
    recommended_movie_names = recommend(selected_movie)
    for i in recommended_movie_names:
        st.subheader(i)

In [40]:
!streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.133.61:8501

  Stopping...
  Stopping...
